In [ ]:
!pip install -r requirements.txt

In [ ]:
!apt-get update
!apt-get install -y libsndfile1-dev

In [ ]:
!python train.py

In [16]:
!aws s3 ls s3-imda-aipo-sagemaker-test/urbansound/raw_data/urbanSound8K/

### Upload data to S3

In [9]:
import os
import sagemaker

import yaml

conf = open("config/conf.yaml", "r")
conf = yaml.safe_load(conf)

params = conf["hyperparameters"]

bucket = conf["s3"]["bucket"]
project_prefix = conf["s3"]["project_prefix"]
raw_data = conf["s3"]["raw_data"]

role = sagemaker.get_execution_role()
sm_sess = sagemaker.session.Session()

# upload data to s3
# raw_data_location = sm_sess.upload_data('/root/sagemaker-keras/data', bucket=bucket, key_prefix=f"{project_prefix}/{raw_data}")
# train_data_location = sm_sess.upload_data('train.csv', bucket=bucket, key_prefix=train_data)
# val_data_location = sm_sess.upload_data('val.csv', bucket=bucket, key_prefix=val_data)

# download data to local
# sm_sess.download_data(".", bucket=bucket, key_prefix=f"{project_prefix}/{raw_data}")

In [20]:
sm_sess.download_data(".", bucket=bucket, key_prefix=f"urbansound/raw_data/UrbanSound8K/audio/UrbanSound8K.csv")

### Create Experiment, Trial & Tracker

In [15]:
from sm_utils import create_experiment, create_trial, delete_experiment

exp = create_experiment("urbansounds")
trial = create_trial("tf-classical-NN", exp.experiment_name)

urbansounds-20211027 experiment created!
tf-classical-NN-20211027 trial created!


In [14]:
delete_experiment("urbansounds")


Experiment urbansounds-tf-20211027 & its components deleted!


In [6]:
from smexperiments.tracker import Tracker

with Tracker.create(display_name='Pre-processing') as tracker:
    tracker.log_parameters(params)
preprocessing_trial_component = tracker.trial_component
# trial.add_trial_component(preprocessing_trial_component)

In [8]:
tracker

In [7]:
preprocessing_trial_component

TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7fe56f162910>,trial_component_name='TrialComponent-2021-10-30-071936-pbzc',display_name='Pre-processing',tags=None,trial_component_arn='arn:aws:sagemaker:ap-southeast-1:052567997892:experiment-trial-component/trialcomponent-2021-10-30-071936-pbzc',response_metadata={'RequestId': '07ac1d61-1381-40e9-a0d2-5afc441f8d57', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '07ac1d61-1381-40e9-a0d2-5afc441f8d57', 'content-type': 'application/x-amz-json-1.1', 'content-length': '134', 'date': 'Sat, 30 Oct 2021 07:19:35 GMT'}, 'RetryAttempts': 0},parameters={'epoch': 12, 'batch_size': 32},input_artifacts={},output_artifacts={},start_time=datetime.datetime(2021, 10, 30, 7, 19, 36, 115748, tzinfo=tzlocal()),status=TrialComponentStatus(primary_status='Completed',message=None),end_time=datetime.datetime(2021, 10, 30, 7, 19, 36, 115817, tzinfo=tzlocal()))

### Start Training

In [12]:
!pwd

/root/sagemaker-keras/project


In [12]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(
    entry_point="sm_entrypoint.sh",
    # dependencies=["train.py", "preprocess.py", "requirements.txt"],
    source_dir=".",
    role=role,
    instance_count=1,
    instance_type="ml.m5.4xlarge",
    framework_version="2.4",
    py_version="py37",
    metric_definitions=[
            {'Name':'train:loss', 'Regex':'loss: ([0-9.]+)'},
            {'Name':'val:loss', 'Regex':'val_loss: ([0-9.]+)'},
            {'Name':'train:accuracy', 'Regex':'accuracy: ([0-9.]+)'},
            {'Name':'val:accuracy', 'Regex':'val_accuracy: ([0-9.]+)'}
        ],
    enable_sagemaker_metrics=True,
    hyperparameters=conf["hyperparameters"]
)

estimator.fit(
    inputs="s3://s3-imda-aipo-sagemaker-test/urbansound/raw_data/UrbanSound8K/audio/fold1/",
    experiment_config={
        "ExperimentName": "urbansounds-20211027",
        "TrialName": "tf-classical-NN-20211027",
        "TrialComponentDisplayName": "Train"
    },
    wait=False # does not print logs, but can launch multiple trials in a for loop
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2021-10-30-08-04-26-246


2021-10-30 08:04:27 Starting - Starting the training job...
2021-10-30 08:04:50 Starting - Launching requested ML instancesProfilerReport-1635581066: InProgress
...
2021-10-30 08:05:22 Starting - Preparing the instances for training.........
2021-10-30 08:06:50 Downloading - Downloading input data...
2021-10-30 08:07:24 Training - Training image download completed. Training in progress..2021-10-30 08:07:25.106869: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-10-30 08:07:25.110001: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-10-30 08:07:25.195279: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2021-10-30 08:07:27,686 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2021-10-30 08:07:2

In [13]:
df_metrics = estimator.training_job_analytics.dataframe()
print(df_metrics)

   timestamp     metric_name     value
0        0.0      train:loss  0.982300
1        0.0  train:accuracy  0.698800
2        0.0    val:accuracy  0.758067
3        0.0        val:loss  0.696500
